<a href="https://colab.research.google.com/github/monishsai4/MyAIStuff/blob/main/Titanic_pred_Logreg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install -q kaggle
#from google.colab import files
#files.upload()
#!mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json
#! kaggle datasets list

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [4]:
#from google.colab import files
#files.upload()

In [5]:
#!kaggle competitions download -c titanic

In [6]:
#!unzip "/content/drive/MyDrive/Colab Notebooks/titanic.zip" -d "/content/drive/MyDrive/Colab Notebooks/Datasets/titanic/"

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import csv



In [8]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/titanic/train.csv')

In [9]:
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/titanic/test.csv')

In [10]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [11]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [12]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [13]:
encoder = LabelEncoder()

train_data.fillna("0", inplace=True)
test_data.fillna("0", inplace=True)

train_data.drop(["Name", "Ticket"], axis=1, inplace=True)
test_data.drop(["Name", "Ticket"], axis=1, inplace=True)

train_labels = train_data.pop("Survived")

for column in train_data.select_dtypes(exclude=["number"]).columns.intersection(test_data.select_dtypes(exclude=["number"]).columns):
    encoder.fit_transform(list(train_data[column]) + list(test_data[column]))

    train_data[column] = encoder.transform(train_data[column])
    test_data[column] = encoder.transform(test_data[column])

In [14]:
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [15]:
print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)

(891, 9)
(891,)
(418, 9)


In [16]:
m_train = train_data.shape[0]
xtrain = train_data.T
xtest = test_data.T
print('test data = ' + str(xtest.shape))
print('train input data = ' + str(xtrain.shape))
ytrain = train_labels.values.reshape(1,-1)
print('train output data = ' + str(ytrain.shape))

test data = (9, 418)
train input data = (9, 891)
train output data = (1, 891)


Lets make this model a n layer neural network.

In [17]:
print(xtrain)

[[-1.73010796 -1.72622007 -1.72233219 ...  1.72233219  1.72622007
   1.73010796]
 [ 0.82737724 -1.56610693  0.82737724 ...  0.82737724 -1.56610693
   0.82737724]
 [ 0.73769513 -1.35557354 -1.35557354 ... -1.35557354  0.73769513
   0.73769513]
 ...
 [-0.50244517  0.78684529 -0.48885426 ... -0.17626324 -0.04438104
  -0.49237783]
 [-0.46489984  1.73514763 -0.46489984 ... -0.46489984  1.13887308
  -0.46489984]
 [ 0.58796609 -1.91264387  0.58796609 ...  0.58796609 -1.91264387
  -0.66233889]]


In [18]:
Layer_dims = [9,4]

In [19]:
m = xtrain.shape[1]
n_x = xtrain.shape[0]
n_h = 4
n_y = ytrain.shape[0]
num_iterations = 10000
learning_rate = 0.054
W1 = np.random.randn(n_h,n_x)*0.01
b1 = np.zeros((n_h,1))
W2 = np.random.randn(n_y,n_h)*0.01
b2 = np.zeros((n_y,1))
cost=np.zeros((num_iterations))

for i in range(0, num_iterations):

    #Forward propagation
    Z1 = np.dot(W1,xtrain) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1) + b2
    A2 = 1/(1+(np.exp(-Z2)))
    logprobs = np.multiply(np.log(A2),ytrain) + np.multiply(np.log(1 - A2),1 - ytrain)
    cost[i] = - np.sum(logprobs)/m

    #backward propagation
    dZ2 = A2 - ytrain
    dW2 = np.dot(dZ2,A1.T)/m
    db2 = np.sum(dZ2,axis = 1,keepdims=True)/m
    dZ1 = np.dot(W2.T,dZ2) * (1 - np.power(A1, 2))
    dW1 = np.dot(dZ1,xtrain.T)/m
    db1 = np.sum(dZ1,axis = 1,keepdims = True)/m

    #updating paramters
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate *db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate *db2

In [20]:
m_train = xtrain.shape[1]
Y_prediction_train = np.zeros((1, m_train))
#w = w.reshape(train_x.shape[0], 1)

Z1 = np.dot(W1,xtrain) + b1
A1 = np.tanh(Z1)
Z2 = np.dot(W2,A1) + b2
A2 = 1/(1+(np.exp(-Z2)))

for i in range(A2.shape[1]):
    if A2[0, i] > 0.5:
        Y_prediction_train[0,i] = 1
    else:
        Y_prediction_train[0,i] = 0
print(Y_prediction_train.shape)
print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - ytrain)) * 100))

(1, 891)
train accuracy: 82.82828282828282 %


In [21]:
#xtest
m_test = xtest.shape[1]
Y_prediction_test = np.zeros((1, m_test))
#w = w.reshape(train_x.shape[0], 1)

Z1 = np.dot(W1,xtest) + b1
A1 = np.tanh(Z1)
Z2 = np.dot(W2,A1) + b2
A2 = 1/(1+(np.exp(-Z2)))

for i in range(A2.shape[1]):
    if A2[0, i] > 0.5:
        Y_prediction_test[0,i] = 1
    else:
        Y_prediction_test[0,i] = 0
print(Y_prediction_test.shape)
#print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - ytrain)) * 100))

(1, 418)


In [22]:
y_pred=Y_prediction_test.reshape(-1,)
print(y_pred.shape)

(418,)


In [25]:
def pred():
  for l in range(0,y_pred.shape[0]):
    print(y_pred[l])
  return

In [34]:
with open('gender_submission.csv', 'w', newline='') as file:
     writer = csv.writer(file)
     #def predtest():
     #for l in range(0,len())
     writer.writerow(["PassengerId", "Survived"])
     m=892
     for x in range(y_pred.shape[0]):

       writer.writerow([m,y_pred[x]])
       m=m+1


In [35]:
gender_sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/gender_submission.csv')

In [36]:
gender_sub.head()

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,0.0
